In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 브라우저 열기
driver = webdriver.Chrome()

# 페이지 접속하기
url = 'https://kovo.co.kr/KOVO/game/v-league?first=%ED%8C%80+%EC%88%9C%EC%9C%84'
driver.get(url)

# 명시적 대기 설정 (최대 10초 대기)
wait = WebDriverWait(driver, 10)

# 여자부 클릭하기 (클릭이 가능할 때까지 대기)
try:
    # 요소가 클릭 가능한 상태일 때까지 대기
    click_1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label:nth-child(2)")))
    click_1.click()
    time.sleep(1)
except Exception as e:
    print("오류 발생:", e)

오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=123.0.6312.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E4C87032+63090]
	(No symbol) [0x00007FF7E4BF2C82]
	(No symbol) [0x00007FF7E4A8EC65]
	(No symbol) [0x00007FF7E4A93D21]
	(No symbol) [0x00007FF7E4A96462]
	(No symbol) [0x00007FF7E4B14062]
	(No symbol) [0x00007FF7E4AF6FDA]
	(No symbol) [0x00007FF7E4B13412]
	(No symbol) [0x00007FF7E4AF6D83]
	(No symbol) [0x00007FF7E4AC83A8]
	(No symbol) [0x00007FF7E4AC9441]
	GetHandleVerifier [0x00007FF7E50825AD+4238317]
	GetHandleVerifier [0x00007FF7E50BF70D+4488525]
	GetHandleVerifier [0x00007FF7E50B79EF+4456495]
	GetHandleVerifier [0x00007FF7E4D60576+953270]
	(No symbol) [0x00007FF7E4BFE54F]
	(No symbol) [0x00007FF7E4BF9224]
	(No symbol) [0x00007FF7E4BF935B]
	(No symbol) [0x00007FF7E4BE9

In [3]:
# 시즌 클릭하기
click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
click_2[0].click()
time.sleep(1)

# ActionChains 객체 생성
actions = ActionChains(driver)

# 방향키 입력 (예: 아래 방향키)
actions.send_keys(Keys.ARROW_DOWN).perform()
time.sleep(1)

# 엔터키 입력
actions.send_keys(Keys.ENTER).perform()
time.sleep(1)

for num in range(5):
    # 라운드 클릭하기
    click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
    click_2[2].click()
    time.sleep(1)
    
    # ActionChains 객체 생성
    actions = ActionChains(driver)
    
    # 방향키 입력 (예: 아래 방향키)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    
    # 엔터키 입력
    actions.send_keys(Keys.ENTER).perform()
    time.sleep(1)
     
    
    click_3 = driver.find_elements(By.CSS_SELECTOR, "button.font-normal")
    c_num = [1, 8, 9]
    
    for numb in range(3):
        for j in c_num:
            click_3[j].click()
            time.sleep(1)
            
            # HTML 다운 받기 & 해석하기
            soup = BeautifulSoup(driver.page_source, 'lxml')
            time.sleep(1)
            
            # 셀 데이터 크롤링
            stat_list = soup.select("tr > td")
            
            stats = []
            for item in stat_list:
                stat = item.text
                data = [stat]
                stats.append(data)
        
            # ".\xa0"값 제거
            cleaned_stats = [item for item in stats if item != ['\xa0']]
        
            # 리스트를 하나로 합치기
            flattened_list = [item for sublist in cleaned_stats for item in sublist]
            list = flattened_list[1:]
        
            # 리스트를 NumPy 배열로 변환
            np_array = np.array(list)
            
            # reshape 사용
            reshaped_array = np_array.reshape(7, -1)
        
            # 데이터프레임으로 전환
            df = pd.DataFrame(reshaped_array)
        
            # globals()를 사용하여 df1, df8, df9와 같은 변수명을 동적으로 생성
            globals()[f"df{j}"] = df
        
        df1.columns = ["공격_순위", "팀", "경기수", "세트수", "공격_시도", "공격_성공", "공격차단", "공격_범실", "공격_성공률"]
        df8.columns =["서브_순위", "팀", "경기수", "세트수", "서브_시도", "서브_성공", "서브_범실", "서브_세트당평균"]
        df9.columns = ["블로킹_순위", "팀", "경기수", "세트수", "블로킹_시도", "블로킹_성공", "블로킹_유효블락", "블로킹_실패", "블로킹_범실", "블로킹_어시스트", "블로킹_세트당평균"]
        df8 = df8.drop(["경기수", "세트수"], axis = 1)
        df9 = df9.drop(["경기수", "세트수"], axis = 1)
                
        # df1과 df8을 팀 기준으로 병합
        merged_df_1_8 = pd.merge(df1, df8, on="팀", how="outer")
        
        # 그 결과와 df9를 팀 기준으로 병합
        season_23_24_df = pd.merge(merged_df_1_8, df9, on="팀", how="outer")
        
        # globals()를 사용하여 round_df0, round_df1, round_df2와 같은 변수명을 동적으로 생성
        globals()[f"round_df{num}"] = season_23_24_df   
        

In [11]:
# 브라우저 열기
driver = webdriver.Chrome()

# 페이지 접속하기
url = 'https://kovo.co.kr/KOVO/game/v-league?first=%ED%8C%80+%EC%88%9C%EC%9C%84'
driver.get(url)

# 명시적 대기 설정 (최대 10초 대기)
wait = WebDriverWait(driver, 10)

# 여자부 클릭하기 (클릭이 가능할 때까지 대기)
try:
    # 요소가 클릭 가능한 상태일 때까지 대기
    click_1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label:nth-child(2)")))
    click_1.click()
    time.sleep(1)
except Exception as e:
    print("오류 발생:", e)

# 시즌 클릭하기
click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
click_2[0].click()
time.sleep(1)

# ActionChains 객체 생성
actions = ActionChains(driver)

# 방향키 입력 (예: 아래 방향키)
actions.send_keys(Keys.ARROW_DOWN).perform()
time.sleep(1)

# 엔터키 입력
actions.send_keys(Keys.ENTER).perform()
time.sleep(1)    

# 라운드 클릭하기
click_2 = driver.find_elements(By.CSS_SELECTOR,"span.ant-select-selection-item")
click_2[2].click()
time.sleep(1)

# ActionChains 객체 생성
actions = ActionChains(driver)

# 방향키 입력 6라운드까지 6번 내리기
for i in range(6):
    actions.send_keys(Keys.ARROW_DOWN).perform()

# 엔터키 입력
actions.send_keys(Keys.ENTER).perform()
time.sleep(1)
 

click_3 = driver.find_elements(By.CSS_SELECTOR, "button.font-normal")

c_num = [1, 8, 9]

for numb in range(3):
    for j in c_num:
        click_3[j].click()
        time.sleep(1)
        
        # HTML 다운 받기 & 해석하기
        soup = BeautifulSoup(driver.page_source, 'lxml')
        time.sleep(1)
        
        # 셀 데이터 크롤링
        stat_list = soup.select("tr > td")
        
        stats = []
        for item in stat_list:
            stat = item.text
            data = [stat]
            stats.append(data)
    
        # ".\xa0"값 제거
        cleaned_stats = [item for item in stats if item != ['\xa0']]
    
        # 리스트를 하나로 합치기
        flattened_list = [item for sublist in cleaned_stats for item in sublist]
        list = flattened_list[84:]
    
        # 리스트를 NumPy 배열로 변환
        np_array = np.array(list)
        
        # reshape 사용
        reshaped_array = np_array.reshape(7, -1)
    
        # 데이터프레임으로 전환
        df = pd.DataFrame(reshaped_array)
    
        # globals()를 사용하여 df1, df8, df9와 같은 변수명을 동적으로 생성
        globals()[f"df{j}"] = df
    
    df1.columns = ["공격_순위", "팀", "경기수", "세트수", "공격_시도", "공격_성공", "공격차단", "공격_범실", "공격_성공률"]
    df8.columns =["서브_순위", "팀", "경기수", "세트수", "서브_시도", "서브_성공", "서브_범실", "서브_세트당평균"]
    df9.columns = ["블로킹_순위", "팀", "경기수", "세트수", "블로킹_시도", "블로킹_성공", "블로킹_유효블락", "블로킹_실패", "블로킹_범실", "블로킹_어시스트", "블로킹_세트당평균"]
    df8 = df8.drop(["경기수", "세트수"], axis = 1)
    df9 = df9.drop(["경기수", "세트수"], axis = 1)
            
    # df1과 df8을 팀 기준으로 병합
    merged_df_1_8 = pd.merge(df1, df8, on="팀", how="outer")
    
    # 그 결과와 df9를 팀 기준으로 병합
    round_df5 = pd.merge(merged_df_1_8, df9, on="팀", how="outer")

In [22]:
# 시즌 컬럼 생성
round_df0["시즌"] = "23-24"
round_df1["시즌"] = "23-24"
round_df2["시즌"] = "23-24"
round_df3["시즌"] = "23-24"
round_df4["시즌"] = "23-24"
round_df5["시즌"] = "23-24"

# 라운드 컬럼 생성
round_df0["라운드"] = 1
round_df1["라운드"] = 2
round_df2["라운드"] = 3
round_df3["라운드"] = 4
round_df4["라운드"] = 5
round_df5["라운드"] = 6

In [23]:
round_23_24 = pd.concat([round_df0, round_df1, round_df2, round_df3, round_df4, round_df5], ignore_index=True)

In [24]:
round_23_24

,공격_순위,팀,경기수,세트수,공격_시도,공격_성공,공격차단,공격_범실,공격_성공률,서브_순위,...,블로킹_순위,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_어시스트,블로킹_세트당평균,시즌,라운드
0,2,GS칼텍스,6,23,785,346,54,56,44.08,2,...,6,280,34,106,95,3,27,1.48,23-24,1
1,6,IBK기업은행,6,24,856,320,46,51,37.38,5,...,4,412,50,166,132,6,43,2.08,23-24,1
2,1,정관장,6,20,652,297,41,54,45.55,1,...,1,315,59,117,89,1,41,2.95,23-24,1
3,7,페퍼저축은행,6,24,789,290,52,49,36.76,6,...,2,386,62,131,125,3,53,2.58,23-24,1
4,3,한국도로공사,6,25,824,344,48,52,41.75,4,...,5,372,45,121,131,11,33,1.8,23-24,1
5,5,현대건설,6,23,734,293,37,46,39.92,7,...,3,386,49,143,119,10,39,2.13,23-24,1
6,4,흥국생명,6,23,800,320,54,57,40.00,3,...,7,287,33,113,91,6,25,1.43,23-24,1
7,2,GS칼텍스,6,24,827,351,64,42,42.44,1,...,7,352,39,136,125,6,31,1.63,23-24,2
8,6,IBK기업은행,6,22,858,320,43,48,37.30,5,...,3,417,51,154,133,4,43,2.32,23-24,2
9,4,정관장,6,25,889,351,56,66,39.48,4,...,6,408,46,170,128,8,36,1.84,23-24,2


In [25]:
round_23_24.to_csv('23-24시즌 라운드별 데이터.csv', index=False)

In [26]:
df_loaded = pd.read_csv('23-24시즌 라운드별 데이터.csv')